In [1]:
# Load in imports

import os
import json
import requests

# we read our API keys from .env - I have a .env.sample for you, just rename to .env
from dotenv import load_dotenv

In [2]:
# Load environment variables from a file called .env

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-y-


In [3]:
# We select our model

MODEL = "gpt-4o-mini"
print(f"Model selected: {MODEL}")

Model selected: gpt-4o-mini


For demo, we will create our own OpenAI request class but in future we will use the OpenAI library.


In [4]:
# We are essentially making a POST request to one endpoint and we create a convenience Class to pass in prompts, temperature and model.

# Temperature - The OpenAI API incorporates a hyperparameter known as temperature that affects the computation of token probabilities when generating output through the large language model. The temperature value ranges from 0 to 2, with lower values indicating greater determinism and higher values indicating more randomness.


class MyCustomOpenAI:

    def __init__(self, system_prompt=None, temperature=0.5, model=MODEL):

        self.model_endpoint = "https://api.openai.com/v1/chat/completions"
        self.temperature = temperature
        self.model = model
        self.system_prompt = system_prompt
        self.api_key = openai_api_key
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}",
        }

    def generate_text(self, prompt):

        payload = {
            "model": self.model,
            "messages": [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt},
            ],
            "stream": False,
            "temperature": self.temperature,
        }

        # Use HTTP POST method from Requests library
        response = requests.post(
            url=self.model_endpoint, headers=self.headers, data=json.dumps(payload)
        ).json()

        return response

In [5]:
# We create an instance of MyCustomOpenAI and pass in a request

client = MyCustomOpenAI(
    model=MODEL,
    system_prompt="You give concise answers to questions with no more than 100 characters",
    temperature=0.0,
)

response = client.generate_text("What is Pydantic?")

In [7]:
print(response)
# print(response["choices"][0])
# print(response["choices"][0]["message"])
# print(response["choices"][0]["message"]["content"])
# print(response["choices"][0]["message"]["content"].replace("\n\n", "\n"))

{'id': 'chatcmpl-As9hvWv6ihbRnJIXu6VrfHejeqU7q', 'object': 'chat.completion', 'created': 1737470391, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Pydantic is a data validation and settings management library for Python.', 'refusal': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 30, 'completion_tokens': 15, 'total_tokens': 45, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'service_tier': 'default', 'system_fingerprint': 'fp_72ed7ab54c'}
